# New nlpfuncs for hathi/digitizedwork

In [2]:
%run ppa_init.ipynb
%run ppa_nlpfuncs.ipynb

## Involving the db

### JSON Fields

In [14]:
def get_or_create_counts(self):
    try:
        wwc = WorkWordCounts.objects.get(work=self)
        countd = wwc.data
    except WorkWordCounts.DoesNotExist:    
        countd = self.calc_counts()
        try:
            wwc, created = WorkWordCounts.objects.get_or_create(work=self, data=countd)
        except Exception as e:
            log.debug(e)
    return countd

def get_counts(self):
    return dict(Counter(self.get_or_create_counts()).most_common())

DigitizedWork.get_or_create_counts = get_or_create_counts
DigitizedWork.get_counts = get_counts

In [15]:
work=random.choice(digiworks)
work

<DigitizedWork: inu.30000035051881>

In [16]:
def cache_all():
    pmap_run(
        get_or_create_counts,
        digiworks,
        num_proc=8,
    )

# cache_all()

### SQL-style: WorkWordCount

In [4]:
def store_counts(self, **opts):
    countd = self.calc_counts()
    workwordcounts = [(self,word,countd[word]) for word in countd]
    pmap_run(do_store_counts, workwordcounts, **opts)

def do_store_counts(workwordcount):
    work,word,count=workwordcount
    try:
        wordobj, word_created = Word.objects.get_or_create(token=word)
        obj, created = WorkWordCount.objects.get_or_create(
            work=work,
            word=wordobj,
            count = count
        )
    except Exception as e:
        logging.debug(e)


DigitizedWork.store_counts = store_counts

In [5]:
work=random.choice(digiworks)
work.author, work.title, work.pub_date

('Monboddo, James Burnett, Lord, 1714-1799',
 'Of the origin and progress of language.',
 1774)

In [6]:
work.store_counts(lim=None, num_proc=8)

Mapping do_store_counts() [x8]: 100%|██████████| 9362/9362 [00:16<00:00, 558.83it/s]


In [21]:
# qset=WorkWordCount.objects.filter(work=work)


In [22]:
# WorkWordCount.objects.filter(word=)

In [23]:
wordobj=Word.objects.get(token='ballad')
wordobj, wordobj.token

(<Word: Word object (1)>, 'ballad')

In [28]:
qset=WorkWordCount.objects.filter(word=wordobj)
{obj.work:obj.count for obj in qset}

{<DigitizedWork: mdp.39015003937433>: 18,
 <DigitizedWork: nyp.33433074393947>: 1,
 <DigitizedWork: chi.76009251>: 2,
 <DigitizedWork: miun.acj8800.0001.001>: 2}